In [1]:
from IPython.display import clear_output
%load_ext autoreload
%autoreload 2
import os
print(os.getcwd())
os.chdir('/home/vco/Projects/pm4py-dcr') # working directory should be pm4py-dcr (the one behind notebooks)
print(os.getcwd())

os.environ['PM4PY_SHOW_EVENT_LOG_DEPRECATION'] = 'false'

/home/vco/Projects/pm4py-dcr/notebooks
/home/vco/Projects/pm4py-dcr


In [3]:
import pm4py
import numpy as np
import pandas as pd
import datetime as dt
import networkx as nx
from pathlib import Path
from copy import deepcopy
from pm4py.util.benchmarking import benchmark_event_log_from_configs
from pm4py.algo.discovery.dcr_discover.algorithm import Variants
from pm4py.objects.dcr import semantics_obj
from datetime import timedelta
from pm4py.algo.evaluation.dcr import algorithm as eval_dcr_on_pdc

pm4py.util.constants.SHOW_EVENT_LOG_DEPRECATION = False
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

# Running example

In [24]:
#test traces for subproceses
dict_keys = ['case:concept:name','concept:name','time:timestamp']

def create_test_traces(ordered_event_matrix):
    start_date = dt.date(2023,1,1)
    case_id = 1
    test_el = []
    for i, trace in enumerate(ordered_event_matrix):
        date = start_date
        for j, event in enumerate(trace):
            test_el.append({dict_keys[0]:str(case_id),dict_keys[1]:str(event),dict_keys[2]:date})
            date = date + dt.timedelta(days=np.random.randint(1, 10)) # TODO: there could be a random sample from a distribution here
        case_id = case_id + 1
    df_test = pd.DataFrame(test_el,columns=['case:concept:name','concept:name','time:timestamp'],dtype=str)
    df_test['time:timestamp'] = pd.to_datetime(df_test['time:timestamp'])
    return pm4py.convert_to_event_log(df_test)

def create_test_traces_with_dates(ordered_event_matrix, ordered_dates_matrix):
    case_id = 1
    test_el = []
    for i, trace in enumerate(ordered_event_matrix):
        for j, event in enumerate(trace):
            date = pd.to_datetime(ordered_dates_matrix[i][j])
            test_el.append({dict_keys[0]:str(case_id),dict_keys[1]:str(event),dict_keys[2]:date})
        case_id = case_id + 1
    df_test = pd.DataFrame(test_el,columns=['case:concept:name','concept:name','time:timestamp'],dtype=str)
    df_test['time:timestamp'] = pd.to_datetime(df_test['time:timestamp'])
    return pm4py.convert_to_event_log(df_test)

A,B,C,D = 'Doctor requests home care','Citizen requests home care','Approved','Denied'
E = 'Consent'
F = 'Municipal assessment'
G = 'Send mail to citizen'
events = [[A,G,E,F,C],
          [A,G,E,F,D],
          [B,E,F,C],
          [B,E,F,D]]
dates = [['2023-01-01','2023-01-02','2023-01-09','2023-01-10','2023-01-17'],
         ['2023-01-01','2023-01-07','2023-01-18','2023-04-01','2023-04-05'],
         ['2023-01-01','2023-01-02','2023-01-15','2023-01-21'],
         ['2023-01-01','2023-01-05','2023-01-20','2023-01-27']]
running_example_trace = create_test_traces_with_dates(events,dates)
tt = pm4py.convert_to_event_log(running_example_trace)
pm4py.write_xes(tt,file_path='/home/vco/Downloads/running_example_trace_doctor.xes')

exporting log, completed traces :: 100%|██████████| 4/4 [00:00<00:00, 13584.79it/s]


In [25]:
configs = [{
            'inBetweenRels': False,
            'timed': False,
            'pending': False,
            'variant': Variants.DCR_BASIC,
        },
    {
            'inBetweenRels': True,
            'timed': True,
            'pending': True,
            'variant': Variants.DCR_SUBPROCESS_ME,
        }
]
dcrs = benchmark_event_log_from_configs(tt,dcr_title="running_example",result_file_prefix="running_example_doctor",configs=configs)
dcrs[1]

[i] Started with config: {'inBetweenRels': False, 'timed': False, 'pending': False, 'variant': <Variants.DCR_BASIC: <module 'pm4py.algo.discovery.dcr_discover.variants.discover_basic' from '/home/vco/Projects/pm4py-dcr/pm4py/algo/discovery/dcr_discover/variants/discover_basic.py'>>}
[i] Mining with basic DisCoveR
[!] Model saved in models/running_example_doctor__config0.xml
[i] Started with config: {'inBetweenRels': True, 'timed': True, 'pending': True, 'variant': <Variants.DCR_SUBPROCESS_ME: <module 'pm4py.algo.discovery.dcr_discover.variants.discover_subprocess_mutual_exclusion' from '/home/vco/Projects/pm4py-dcr/pm4py/algo/discovery/dcr_discover/variants/discover_subprocess_mutual_exclusion.py'>>}
[i] Mining with Sp-DisCoveR (ME)
[!] Model saved in models/running_example_doctor_T_config1.xml


{'events': {'Approved',
  'Citizen requests home care',
  'Consent',
  'Consent_ne',
  'Denied',
  'Doctor requests home care',
  'Municipal assessment',
  'Municipal assessment_ne',
  'S0',
  'S1',
  'S1_ne',
  'Send mail to citizen'},
 'conditionsFor': {'Consent': {'S1_ne'},
  'S0': {'Municipal assessment_ne'},
  'Send mail to citizen': {'S1_ne'},
  'S1': set(),
  'Municipal assessment': {'Consent_ne'}},
 'milestonesFor': {},
 'responseTo': {'Consent': {'Municipal assessment'},
  'S0': set(),
  'Send mail to citizen': {'Consent'},
  'S1': {'Consent'},
  'Municipal assessment': {'S0'},
  'Doctor requests home care': {'Send mail to citizen'}},
 'includesTo': {'Consent': set(),
  'S0': set(),
  'Send mail to citizen': set(),
  'S1': set(),
  'Municipal assessment': set()},
 'excludesTo': {'Consent': {'Consent'},
  'S0': {'S0'},
  'Send mail to citizen': {'Send mail to citizen'},
  'S1': {'S1'},
  'Municipal assessment': {'Municipal assessment'},
  'Citizen requests home care': {'Send ma

In [333]:
from pm4py.algo.evaluation.simplicity.variants import dcr_relations as dcr_simplicity

In [340]:
len(dcrs[0]['events'])

7

In [335]:
dcr_simplicity.get_simplicity(dcrs[0])

(28, 0)

In [339]:
len(dcrs[1]['events'])

12

In [336]:
dcr_simplicity.get_simplicity(dcrs[1])

(17, 5)

# PDC Challanges 19 - 22

## Do the F1 score based on PDC challenges
location: Datasets/PDC19,PDC20,PDC21,PDC22
inside: Ground Truth Logs | Test Logs | Training Logs
exceptions: PDC20 have Models (pnml) and PDC22 has Base Logs

In [33]:
results = eval_dcr_on_pdc.score_everything()

[i] Started for PDC19
[i] Log pdc_2019_4.xes
[i] Mining with basic DisCoveR
tp: 47| fp: 4 | tn: 38 | fn: 1
[i] Done for PDC19
[i] Mining with Sp-DisCoveR (ME)
tp: 47| fp: 4 | tn: 38 | fn: 1
[i] Done for PDC19
[i] Mining with Sp-DisCoveR (ME)
tp: 47| fp: 4 | tn: 38 | fn: 1
[i] Done for PDC19
[i] Log pdc_2019_1.xes
[i] Mining with basic DisCoveR
tp: 45| fp: 0 | tn: 45 | fn: 0
[i] Done for PDC19
[i] Mining with Sp-DisCoveR (ME)
tp: 45| fp: 0 | tn: 45 | fn: 0
[i] Done for PDC19
[i] Mining with Sp-DisCoveR (ME)
tp: 45| fp: 0 | tn: 45 | fn: 0
[i] Done for PDC19
[i] Log pdc_2019_7.xes
[i] Mining with basic DisCoveR
tp: 45| fp: 7 | tn: 38 | fn: 0
[i] Done for PDC19
[i] Mining with Sp-DisCoveR (ME)
tp: 45| fp: 8 | tn: 37 | fn: 0
[i] Done for PDC19
[i] Mining with Sp-DisCoveR (ME)
tp: 45| fp: 9 | tn: 36 | fn: 0
[i] Done for PDC19
[i] Log pdc_2019_2.xes
[i] Mining with basic DisCoveR
tp: 45| fp: 0 | tn: 45 | fn: 0
[i] Done for PDC19
[i] Mining with Sp-DisCoveR (ME)
tp: 45| fp: 0 | tn: 45 | fn: 0


In [34]:
results

,PDC Year,Log name,Algorithm,TP,FP,TN,FN,F1-PDC,F1,BAC,Training Fitness,#Relations,#Subprocesses,#InSpActivities,#Activities,Runtime
0,PDC19,pdc_2019_4.xes,DisCoveR,47,4,38,1,0.940495,0.949495,0.941964,1.0,382,0,0,34,0.035991
1,PDC19,pdc_2019_4.xes,DisCoveR_Tics,47,4,38,1,0.940495,0.949495,0.941964,1.0,233,4,14,38,0.067809
2,PDC19,pdc_2019_4.xes,DisCoveR_Tics_no_re,47,4,38,1,0.940495,0.949495,0.941964,1.0,107,4,14,38,0.066389
3,PDC19,pdc_2019_1.xes,DisCoveR,45,0,45,0,1.000000,1.000000,1.000000,1.0,255,0,0,45,0.152655
4,PDC19,pdc_2019_1.xes,DisCoveR_Tics,45,0,45,0,1.000000,1.000000,1.000000,1.0,127,8,20,53,0.178542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177,PDC22,pdc2022_120010.xes,DisCoveR_Tics,158,12,488,342,0.477424,0.471642,0.646000,1.0,73,3,6,20,0.060345
1178,PDC22,pdc2022_120010.xes,DisCoveR_Tics_no_re,172,19,481,328,0.506781,0.497829,0.653000,1.0,56,3,6,20,0.059425
1179,PDC22,pdc2022_120001.xes,DisCoveR,175,30,470,325,0.510078,0.496454,0.645000,1.0,63,0,0,14,0.031045
1180,PDC22,pdc2022_120001.xes,DisCoveR_Tics,191,32,468,309,0.542568,0.528354,0.659000,1.0,52,2,4,16,0.054284


In [16]:
configs = [{
            'inBetweenRels': False,
            'timed': False,
            'pending': False,
            'variant': Variants.DCR_BASIC,
            'alg_name': 'DisCoveR'
        },
        {
            'inBetweenRels': True,
            'timed': False,
            'pending': True,
            'nestings': False,
            'variant': Variants.DCR_SUBPROCESS_ME,
            'alg_name': 'Nst-DisCoveR'
        }
]

results = eval_dcr_on_pdc.score_everything(folders=['PDC19'],configs=configs)

[i] Started for PDC19
[i] Log pdc_2019_4.xes
[i] Mining with basic DisCoveR
tp: 47| fp: 4 | tn: 38 | fn: 1
[i] Done for PDC19
[i] Mining with Sp-DisCoveR (ME)
tp: 47| fp: 4 | tn: 38 | fn: 1
[i] Done for PDC19
[i] Log pdc_2019_1.xes
[i] Mining with basic DisCoveR
tp: 45| fp: 0 | tn: 45 | fn: 0
[i] Done for PDC19
[i] Mining with Sp-DisCoveR (ME)
tp: 45| fp: 0 | tn: 45 | fn: 0
[i] Done for PDC19
[i] Log pdc_2019_7.xes
[i] Mining with basic DisCoveR
tp: 45| fp: 7 | tn: 38 | fn: 0
[i] Done for PDC19
[i] Mining with Sp-DisCoveR (ME)


KeyboardInterrupt: 

# Mine from a few BPI logs
see notebook 7